In [ ]:

'''
For the fine-tuning of the VGG-Face network for the emotion
recognition task, we investigated various options in our preliminary
analysis. We found that combining weight decay and dropout for regularization
gives the best results on the FER validation set. We carry
out a multi-stage fine-tuning. In the first stage, we fine-tune on the
FER public test set, and run weight updates for five epochs. In the second
stage, we update the upper layers (higher than layer 27) using'''

''' We then fine-tune the VGG-face model on FER 2013
dataset, using both the training and the public test set; during
training we use data augmentation by jittering the scale, flipping
and rotating the faces. The aim is to make the network more robust
to small misalignment of the faces. We also apply a strong dropout
on the last layer of the VGG (keeping only 5% of the nodes) to
prevent over-fitting. We achieve a performance of 71.2% on the
FER private test set, which is slightly higher than the previously
published results '''

'''During the training of the deep networks, we oversample the training
images by rotating them around their center by a random angle between 
−15° and 15°, and by circularly shifting the images in the horizontal and 
vertical directions by an amount no more than 20% of the image size. 
This approach helps our network to be more robust against alignment errors. 
In Fig. 2, we show the training curves of two stages of fine-tuning of the 
network with the FER dataset, where we set the learning rate and weight 
decay to 0.0005, momentum to 0.9, and dropout probability to 0.8 [25].'''


### Function to train the model

In [ ]:
def train_model(network, criterion, optimizer, trainLoader, valLoader, n_epochs = 10, use_gpu = False):
    if use_gpu:
        network = network.cuda()
        criterion = criterion.cuda()
        
    t_loss, t_acc, v_loss, v_acc = (np.zeros(n_epochs) for i in range(4))    
    
    # Training loop.
    for epoch in range(0, n_epochs):
        correct = 0.0
        cum_loss = 0.0
        counter = 0

        # Make a pass over the training data.
        t = tqdm(trainLoader, desc = 'Training epoch %d' % epoch)
        network.train()  # This is important to call before training!
        for (i, (inputs, labels)) in enumerate(t):

            # Wrap inputs, and targets into torch.autograd.Variable types.
            inputs = Variable(inputs)
            labels = Variable(labels)
            
            if use_gpu:
                inputs = inputs.cuda()
                labels = labels.cuda()

            # Forward pass:
            outputs = network(inputs)
            loss = criterion(outputs, labels)

            # Backward pass:
            optimizer.zero_grad()
            # Loss is a variable, and calling backward on a Variable will
            # compute all the gradients that lead to that Variable taking on its
            # current value.
            loss.backward() 

            # Weight and bias updates.
            optimizer.step()

            # logging information.
            cum_loss += loss.data[0]
            max_scores, max_labels = outputs.data.max(1)
            correct += (max_labels == labels.data).sum()
            counter += inputs.size(0)
            t.set_postfix(loss = cum_loss / (1 + i), accuracy = 100 * correct / counter)
            
        t_loss[epoch] = (cum_loss/len(t))
        t_acc[epoch] = (100*correct/counter)

        # Make a pass over the validation data.
        correct = 0.0
        cum_loss = 0.0
        counter = 0
        t = tqdm(valLoader, desc = 'Validation epoch %d' % epoch)
        network.eval()  # This is important to call before evaluating!
        for (i, (inputs, labels)) in enumerate(t):

            # Wrap inputs, and targets into torch.autograd.Variable types.
            inputs = Variable(inputs)
            labels = Variable(labels)
            
            if use_gpu:
                inputs = inputs.cuda()
                labels = labels.cuda()

            # Forward pass:
            outputs = network(inputs)
            loss = criterion(outputs, labels)

            # logging information.
            cum_loss += loss.data[0]
            max_scores, max_labels = outputs.data.max(1)
            correct += (max_labels == labels.data).sum()
            counter += inputs.size(0)
            t.set_postfix(loss = cum_loss / (1 + i), accuracy = 100 * correct / counter)
            
        v_loss[epoch] = (cum_loss/len(t))
        v_acc[epoch] = (100*correct/counter)
        
                
    lab_utils.generate_plots(t_loss, v_loss, t_acc, v_acc, n_epochs)
            

### VGG16-Face model

In [33]:
import VGG_FACE
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.transforms as transforms

ImportError: No module named torch

In [44]:
import sys
sys.executable


'/Users/crystalgong/anaconda/bin/python2'

In [ ]:
model = VGG_FACE.VGG_FACE

model.load_state_dict(torch.load('VGG_FACE.pth'))

#how to test whether this is pretrained?
#model.eval()


### Dataset: FERplus

In [32]:
import os
from skimage import io
import pandas as pd
import numpy as np
import argparse
import torch
from torch.utils.data import Dataset, DataLoader

ImportError: No module named torch

In [24]:
class FaceEmotionsDataset(Dataset):

    def __init__(self, csv_file, root_dir):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
        """
        self.emotions_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir

    def __len__(self):
        return len(self.emotions_frame)

    def __getitem__(self, idx):
        img_name = emotions_frame.iloc[idx][1]
        img_path = os.path.join('all-data/FERPlus/data/FER2013Train', img_name)
        image = io.imread(img_path)
        #this takes the most highest ranked emotion. if two emotions have the same ranking, it just takes the first one
        emotion = np.argmax(emotions_frame.iloc[idx,2:].as_matrix())
        sample = {'image': image, 'emotion': emotion}
        sample

NameError: name 'Dataset' is not defined

In [23]:
face_emotions = FaceEmotionsDataset(csv_file = 'all-data/FERPlus/fer2013new.csv', 
                                    root_dir = 'all-data/FERPlus/data/FER2013Train')

NameError: name 'FaceEmotionsDataset' is not defined

In [22]:
emotions_frame = pd.read_csv('all-data/FERPlus/fer2013new.csv')

print(len(emotions_frame))

#def __len__(self):
len(emotions_frame)

#def __getitem__(self, idx):
img_name = emotions_frame.iloc[idx][1]
img_path = os.path.join('all-data/FERPlus/data/FER2013Train', img_name)
image = io.imread(img_path)
#this takes the most highest ranked emotion. if two emotions have the same ranking, it just takes the first one
emotion = np.argmax(emotions_frame.iloc[idx,2:].as_matrix())
sample = {'image': image, 'emotion': emotion}
sample

35887


{'emotion': 4, 'image': array([[ 30,  24,  21, ...,  37,  44,  37],
        [ 31,  22,  21, ...,  37,  35,  41],
        [ 27,  22,  19, ...,  33,  34,  40],
        ..., 
        [ 29,  29,  26, ..., 118, 132, 148],
        [ 30,  30,  27, ..., 154, 159, 166],
        [ 32,  29,  28, ..., 172, 173, 173]], dtype=uint8)}

In [ ]:
emotions_dataset = FaceEmotionsDataset(

In [ ]:

emotions = pd.read_csv('all-data/FERPlus/fer2013new.csv')
rows = open('all-data/FERPlus/fer2013new.csv').read().strip().split("\n")


In [ ]:

#separate out the labels
names = [r.split(",")[:2] for r in rows]
classes = [r.split(",")[2:] for r in rows]
#get the max label
maxClass = [np.argmax(c) for c in classes]
#put them back together
training = []
validation = []
test = []
zipped =zip(names, maxClass)



In [ ]:
#THIS CODE IS ADAPTED FROM https://github.com/oarriaga/face_classification/tree/master/src/utils
#WE DID NOT WRITE IT
class DataManager(object):
    """Class for loading fer2013 emotion classification dataset or
        imdb gender classification dataset."""
    def __init__(self, dataset_path=None, image_size=(48, 48)):
        self.dataset_path = dataset_path
        self.image_size = image_size
        if self.dataset_path != None:
            self.dataset_path = dataset_path

    def get_data(self):
        ground_truth_data = self._load_fer2013()
        return ground_truth_data

    def _load_fer2013(self):
        data = pd.read_csv(self.dataset_path)
        pixels = data['pixels'].tolist()
        width, height = 48, 48
        faces = []
        for pixel_sequence in pixels:
            face = [int(pixel) for pixel in pixel_sequence.split(' ')]
            face = np.asarray(face).reshape(width, height)
            face = cv2.resize(face.astype('uint8'), self.image_size)
            faces.append(face.astype('float32'))
        faces = np.asarray(faces)
        faces = np.expand_dims(faces, -1)
        emotions = pd.get_dummies(data['emotion']).as_matrix()
        return faces, emotions

def get_labels(dataset_name):
    return {0:'angry',1:'disgust',2:'fear',3:'happy', 4:'sad',5:'surprise',6:'neutral'}

def get_class_to_arg(dataset_name='fer2013'):
    return {'angry':0, 'disgust':1, 'fear':2, 'happy':3, 'sad':4, 'surprise':5, 'neutral':6}
   
def split_data(x, y, validation_split=.2):
    num_samples = len(x)
    num_train_samples = int((1 - validation_split)*num_samples)
    train_x = x[:num_train_samples]
    train_y = y[:num_train_samples]
    val_x = x[num_train_samples:]
    val_y = y[num_train_samples:]
    train_data = (train_x, train_y)
    val_data = (val_x, val_y)
    return train_data, val_data

In [ ]:
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor

imgTransform = transforms.Compose([transforms.Scale(256), #scale to 256x256
                                   transforms.CenterCrop(224), #crops the image at center to 224x224
                                   transforms.ToTensor()])
                                   #, #turn the jpg/pil/wahtever image into a tensor
                                   #transforms.Normalize(mean = [0.485, 0.456, 0.406], #normalize with these vals
                                                        #std=[0.229, 0.224, 0.225])])
                                    ##HOW TO GET NORMALIZED VALUES?
                                    #to add: jitter/rotate data augmentation, flipping, 
                                   
#this doesn't work because the data is organized w a csv file w prob distrib of labels 
#instead of a single ground truth
#see this paper: https://arxiv.org/pdf/1608.01041.pdf
trainset = ImageFolder(root= './all-data/FERPlus/data/FER2013Train/', transform = imgTransform) 
valset = ImageFolder(root='./all-data/FERPlus/data/FER2013Valid/', transform = imgTransform)

trainLoader = torch.utils.data.DataLoader(trainset, batch_size = 64, 
                                          shuffle = True, num_workers = 0)
valLoader = torch.utils.data.DataLoader(valset, batch_size = 64, 
                                       shuffle = False, num_workers = 0)

### set learning rate, loss, optimizer, all variable stuff

In [ ]:
#"where we set the learning rate and weight decay to 0.0005, momentum to 0.9, and dropout probability to 0.8 [25]"
learningRate = 5e-4



# Definition of our network.
#how to change the last fc layer of the model to nn.linear(4096, 7) instead of (4096, 2622)?
model.fc = nn.Linear(512, 2)

#Definition of our loss. #maybe need to change this?
criterion = nn.CrossEntropyLoss()

# Definition of optimization strategy. # maybe need to change this?
optimizer = optim.SGD(network.parameters(), lr = learningRate)

train_model(model, criterion, optimizer, trainLoader, valLoader, n_epochs = 3, use_gpu = True)